In [1]:
using Random
rng = Random.default_rng()

TaskLocalRNG()

In [2]:
ENV["JULIA_NUM_THREADS"] = 8

8

In [2]:
Threads.nthreads()

8

---
# Bulid an ANN model

---

In [3]:
using Lux
nn_model = Lux.Chain(
    Lux.Dense(1, 8, tanh), 
    Lux.Dense(8, 8, tanh), 
    Lux.Dense(8, 1)
    )

Chain(
    layer_1 = Dense(1 => 8, tanh_fast),  # 16 parameters
    layer_2 = Dense(8 => 8, tanh_fast),  # 72 parameters
    layer_3 = Dense(8 => 1),            # 9 parameters
)         # Total: 97 parameters,
          #        plus 0 states.

In [4]:
using Random, Plots
using DifferentialEquations
# using Optimization, Lux, DiffEqFlux,  OptimizationOptimJL

mumax(T) = -0.00266071 .* T .^ 2 .+ 0.18037857 .* T .- 2.86700000
xmax(T) = 0.76071429 .* T .^ 2 .- 55.69857143 .* T .+ 1058.18000000
pmax(T) = 0.57678571 .* T .^ 2 .- 42.06642857 .* T .+ 838.23200000
ypx(T) = -0.02071429 .* T .^ 2 .+ 1.90357143 .* T .- 26.44400000
yx(T) = -0.00016071 .* T .^ 2 .+ 0.00917857 .* T .- 0.09160000

t = (0.0, 33.0)
ts = collect(0.0:0.1:t[2])
u0 = [2.75; 150; 0; 30]

rng = Random.default_rng()

function ethanol(dy, y, p, t)
    h, cp, d, frec, v, ks, ki, m, n, mx, mp, Tsp = [-511 3.82 1060 31 59.66 4.1 4e-3 1.0 1.5 0.2 0.1 30.0]
    X = y[1]
    S = y[2]
    P = y[3]
    T = y[4]

    dy[1] = mumax(T) * (S / (S + ks)) * real((1 - (X / xmax(T)) + 0im)^m) #* real(((1 - (P / pmax(T))) + 0im)^(n) * X) * ℯ^((-1) * ki * S)# *(ℯ)^(-ki*S) * ((1-(X/xmax(T)))^m) * (((1-(P/pmax(T)))^n)*X)
    dy[2] = -((dy[1] / yx(T)) + (mx * X))
    dy[3] = (ypx(T) * dy[1]) + (mp * X)

    # for the version with a controller, uncomment this
    if T ≤ Tsp
        dy[4] = (h / (d * cp)) * dy[2]
    else
        dy[4] = (h / (d * cp)) * dy[2] - (frec / v) * (T - Tsp)
    end
end

prob = DifferentialEquations.ODEProblem(ethanol, u0, t, 0)
# https://docs.sciml.ai/DiffEqDocs/stable/basics/common_solver_opts/
sol = DifferentialEquations.solve(prob, Tsit5(), saveat=ts)  # saveat=ts to save more points

retcode: Success
Interpolation: 1st order linear
t: 331-element Vector{Float64}:
  0.0
  0.1
  0.2
  0.3
  0.4
  0.5
  0.6
  0.7
  0.8
  0.9
  1.0
  1.1
  1.2
  ⋮
 31.9
 32.0
 32.1
 32.2
 32.3
 32.4
 32.5
 32.6
 32.7
 32.8
 32.9
 33.0
u: 331-element Vector{Vector{Float64}}:
 [2.75, 150.0, 0.0, 30.0]
 [2.7640614825642875, 149.58528621672951, 0.19683190652817298, 30.051000982866867]
 [2.7782105526391008, 149.16782885695707, 0.39532389311487376, 30.09975692349459]
 [2.7924418335954675, 148.74776374578562, 0.5953955272395806, 30.1463647288178]
 [2.8067502814798897, 148.32521904570183, 0.7969705689768467, 30.19091741555231]
 [2.821131168911509, 147.90031561786003, 0.9999768023572377, 30.23350424241675]
 [2.835580044822556, 147.4731679481131, 1.2043455629895297, 30.274211144680095]
 [2.8500927842833517, 147.04388288987894, 1.4100124614494765, 30.31311983935802]
 [2.864665522327556, 146.61256115198773, 1.6169166125574232, 30.350308474208855]
 [2.879294601528852, 146.17929852103828, 1.82499998

In [6]:
nn_param, nn_rebulid = Lux.destructure(nn_model)
# nn_param
# nn_rebulid(nn_param)

LoadError: UndefVarError: `destructure` not defined

---
# Define a OED Problem
---

In [7]:
using DifferentialEquations

mumax(T) = -0.00266071 .* T .^ 2 .+ 0.18037857 .* T .- 2.86700000
xmax(T) = 0.76071429 .* T .^ 2 .- 55.69857143 .* T .+ 1058.18000000
pmax(T) = 0.57678571 .* T .^ 2 .- 42.06642857 .* T .+ 838.23200000
ypx(T) = -0.02071429 .* T .^ 2 .+ 1.90357143 .* T .- 26.44400000
yx(T) = -0.00016071 .* T .^ 2 .+ 0.00917857 .* T .- 0.09160000

function ethanol(dy, y, p, t)
    h, cp, d, frec, v, ks, ki, m, n, mx, mp, Tsp = [-511 3.82 1060 31 59.66 4.1 4e-3 1.0 1.5 0.2 0.1 30.0]
    X = y[1]
    S = y[2]
    P = y[3]
    T = y[4]
    #* real(((1 - (P / pmax(T))) + 0im)^(n) * X) * ℯ^((-1) * ki * S)# *(ℯ)^(-ki*S) * ((1-(X/xmax(T)))^m) * (((1-(P/pmax(T)))^n)*X)
    dy[1] = mumax(T) * (S / (S + ks)) * real((1 - (X / xmax(T)) + 0im)^m) 
    dy[2] = -((dy[1] / yx(T)) + (mx * X))
    dy[3] = (ypx(T) * dy[1]) + (mp * X)
    dy[4] = (h / (d * cp)) * dy[2] - (frec / v) * nn_rebulid(p)([t])[1]
end

y0 = [2.75f0, 150.0f0, 0.0f0, 30.0f0]
tspan = (0.0f0, 33.0f0)
ts = Float32.(collect(0.0:0.01:tspan[2]))
prob = DifferentialEquations.ODEProblem(ethanol, y0, tspan, nn_param)
DifferentialEquations.solve(prob, Vern9(), abstol = 1e-10, reltol = 1e-10) 

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 190902-element Vector{Float32}:
  0.0
  0.00027697452
  0.00044262107
  0.0006005671
  0.0007513083
  0.0008578754
  0.000979481
  0.0010897384
  0.0011861374
  0.0012750039
  0.0013623905
  0.0014523776
  0.0015432885
  ⋮
 32.979897
 32.98166
 32.983467
 32.98529
 32.987167
 32.9889
 32.99098
 32.99252
 32.99449
 32.9963
 32.9982
 33.0
u: 190902-element Vector{Vector{Float32}}:
 [2.75, 150.0, 0.0, 30.0]
 [2.7500389, 149.99886, 0.00054280437, 30.000145]
 [2.750062, 149.99817, 0.0008674388, 30.00023]
 [2.7500842, 149.99751, 0.001176986, 30.000313]
 [2.7501054, 149.99689, 0.0014724171, 30.000391]
 [2.7501204, 149.99644, 0.0016812757, 30.000446]
 [2.7501373, 149.99594, 0.0019196104, 30.00051]
 [2.7501528, 149.99548, 0.0021357061, 30.000566]
 [2.7501664, 149.99509, 0.0023246421, 30.000616]
 [2.7501788, 149.99472, 0.0024988162, 30.000662]
 [2.750191, 149.99435, 0.0026700913, 30.000708]
 [2.7502036, 149.99399, 0.0028

---
# Define Optimization Problem
---

In [8]:
using SciMLSensitivity
function predict_adjoint(θ)
    Array(
        SciMLSensitivity.solve(prob, Vern9(), p = θ, saveat = ts,
        sensealg = InterpolatingAdjoint())
        )
end

using Statistics
function loss_adjoint(θ)
    x = predict_adjoint(θ)
    mean(abs2, 30.0 .- x[4, :]) +
    mean(abs2, [first(nn_rebulid(θ)([t])) for t in ts]) / 10
end

loss_nn_param = loss_adjoint(nn_param)

111.01563932387955

In [9]:
using Plots
callback = function (param, loss; doplot = false)
    println(loss)

    if doplot
        p = plot(solve(remake(prob, p = param), Tsit5(), saveat = 0.01), ylim = (-6, 6), lw = 3)
        plot!(p, ts, [first(nn_rebulid(param)([t])) for t in ts], label = "u(t)", lw = 3)
        display(p)
    end

    return false
end

# Display the ODE with the current parameter values.
callback(nn_param, loss_nn_param)

111.01563932387955


false

In [10]:
using Optimization, OptimizationFlux
# Setup and run the optimization
# loss1 = loss_adjoint(nn_param)
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x, p) -> loss_adjoint(x), adtype)

(::OptimizationFunction{true, AutoZygote, var"#10#11", Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED_NO_TIME), Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing}) (generic function with 1 method)

In [11]:
optprob = Optimization.OptimizationProblem(optf, nn_param)

OptimizationProblem. In-place: true
u0: 97-element Vector{Float32}:
  0.80449843
  0.7833807
 -0.26423168
 -0.14755043
  0.52638245
  0.6362073
 -0.6610261
  0.25659603
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
 -0.6290883
 -0.39728975
 -0.7119011
  0.7430587
  0.36303928
  0.6880909
  0.45844764
 -0.34602368
  0.0

In [12]:
res1 = Optimization.solve(optprob, ADAM(0.005), callback = callback, maxiters = 100)

┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21
Enzyme compilation failed due to illegal type analysis.
Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__10121({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* 


Illegal updateAnalysis prev:{[-1]:Pointer, [-1,0]:Integer, [-1,1]:Integer, [-1,2]:Integer, [-1,3]:Integer, [-1,4]:Integer, [-1,5]:Integer, [-1,6]:Integer, [-1,7]:Integer} new: {[-1]:Pointer, [-1,0]:Float@double}
val:   %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104 origin=  %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104

Caused by:
Stacktrace:
 [1] Float64
   @ ./float.jl:159
 [2] convert
   @ ./number.jl:7
 [3] setindex!
   @ ./array.jl:971
 [4] hvcat_fill!
   @ ./abstractarray.jl:2135



111.01563932387955

┌ Warning: ReverseDiffVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:61
ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float32, Nothing} to Int64 is not defined. Please use `ReverseDiff.value` instead.

┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21
Enzyme compilation failed due to illegal type analysis.
Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__12289({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* 

...

┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21
Enzyme compilation failed due to illegal type analysis.
Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__14689({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* 



3.429863376734533


Illegal updateAnalysis prev:{[-1]:Pointer, [-1,0]:Integer, [-1,1]:Integer, [-1,2]:Integer, [-1,3]:Integer, [-1,4]:Integer, [-1,5]:Integer, [-1,6]:Integer, [-1,7]:Integer} new: {[-1]:Pointer, [-1,0]:Float@double}
val:   %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104 origin=  %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104

Caused by:
Stacktrace:
 [1] Float64
   @ ./float.jl:159
 [2] convert
   @ ./number.jl:7
 [3] setindex!
   @ ./array.jl:971
 [4] hvcat_fill!
   @ ./abstractarray.jl:2135
┌ Warning: ReverseDiffVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:61
ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float32, Nothing} to Int64 is not defined. Please use `ReverseDiff.value` instead.

┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21
Enzyme compilation failed due to illegal type analysis.
Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__14849({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* 


Illegal updateAnalysis prev:{[-1]:Pointer, [-1,0]:Integer, [-1,1]:Integer, [-1,2]:Integer, [-1,3]:Integer, [-1,4]:Integer, [-1,5]:Integer, [-1,6]:Integer, [-1,7]:Integer} new: {[-1]:Pointer, [-1,0]:Float@double}
val:   %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104 origin=  %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104

Caused by:
Stacktrace:
 [1] Float64
   @ ./float.jl:159
 [2] convert
   @ ./number.jl:7
 [3] setindex!
   @ ./array.jl:971
 [4] hvcat_fill!
   @ ./abstractarray.jl:2135
┌ Warning: ReverseDiffVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity ~/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:61
ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float32, Nothing} to Int64 is not defined. Please use `ReverseDiff.value` instead.

In [11]:
using OptimizationNLopt
optprob2 = Optimization.OptimizationProblem(optf, res1.u)
res2 = Optimization.solve(optprob2, NLopt.LD_LBFGS(), maxiters = 100)

In [ ]:
function loss_adjoint(θ)
    x = predict_adjoint(θ)
    mean(abs2, 30.0 .- x[4, :]) +
    mean(abs2, [first(nn_rebulid(θ)([t])) for t in ts])
end
optf3 = Optimization.OptimizationFunction((x, p) -> loss_adjoint(x), adtype)

optprob3 = Optimization.OptimizationProblem(optf3, res2.u)
res3 = Optimization.solve(optprob3, NLopt.LD_LBFGS(), maxiters = 100)

In [ ]:
l = loss_adjoint(res3.u)
callback(res3.u, l)
p = plot(solve(remake(prob, p = res3.u), Tsit5(), saveat = 0.01), ylim = (-6, 6), lw = 3)
plot!(p, ts, [first(ann_rebulid(res3.u)([t])) for t in ts], label = "u(t)", lw = 3)